# **Assignment - 2**

The Marketing department of Adventure Works Cycles wants to increase sales by targeting specific
customers for a mailing campaign. The company's database contains a list of past customers and a list of
potential new customers. By investigating the attributes of previous bike buyers, the company hopes to
discover patterns that they can then apply to potential customers. They hope to use the discovered patterns
to predict which potential customers are most likely to purchase a bike from Adventure Works Cycles. 

Part I: Based on Feature Selection, Cleaning, and Preprocessing to Construct an Input from Data Source  
  
(a) Examine the values of each attribute and Select a set of attributes only that would affect to predict
future bike buyers to create your input for data mining algorithms. Remove all the unnecessary
attributes. (Select features just by analysis). 


###  Selected Features  
- Gender  
- MaritalStatus  
- YearlyIncome  
- Education  
- Occupation  
- HomeOwnerFlag  
- NumberCarsOwned  
- NumberChildrenAtHome  
- TotalChildren   
- Age  
- AvgMonthSpend


###  Dropped Features  
- CustomerID   
- Title  
- FirstName  
- MiddleName  
- LastName  
- Suffix  
- AddressLine1  
- AddressLine2  
- City  
- StateProvince   
- CountryRegion  
- PostalCode 
- PhoneNumber
- BirthDate  

###  Target Variable
- BikeBuyer  

(b) Create a new Data Frame with the selected attributes only. 


In [35]:
import pandas as pd

customers = pd.read_csv("Data/AWcustomers.csv")
sales = pd.read_csv("Data/AWsales.csv")

df = pd.merge(customers, sales, on="CustomerID", how="inner")

# BirthDate -> Age
df["BirthDate"] = pd.to_datetime(df["BirthDate"])
df["Age"] = (pd.Timestamp("today") - df["BirthDate"]).dt.days // 365


selected_features = [
    "Age", "Gender", "MaritalStatus", "Education", "Occupation", "HomeOwnerFlag", "NumberCarsOwned", "NumberChildrenAtHome", "TotalChildren", "YearlyIncome", "AvgMonthSpend", "BikeBuyer"
]


df_selected = df[selected_features]
print(df_selected.head())

   Age Gender MaritalStatus        Education      Occupation  HomeOwnerFlag  \
0   37      M             M        Bachelors        Clerical              1   
1   53      M             M  Partial College        Clerical              1   
2   39      F             S        Bachelors        Clerical              0   
3   47      M             M  Partial College  Skilled Manual              1   
4   50      M             S  Partial College  Skilled Manual              1   

   NumberCarsOwned  NumberChildrenAtHome  TotalChildren  YearlyIncome  \
0                3                     0              1         81916   
1                2                     1              2         81076   
2                3                     0              0         86387   
3                2                     1              2         61481   
4                1                     0              0         51804   

   AvgMonthSpend  BikeBuyer  
0          50.97          1  
1          53.11          

(c) Determine a Data value type (Discrete, or Continuous, then Nominal, Ordinal, Interval, Ratio) of
each attribute in your selection to identify preprocessing tasks to create input for your data mining.

In [36]:
Nominal = [
    'CountryRegionName',
    'Occupation',
    'Gender',
    'MaritalStatus'
]

Ordinal = [
    'Education'  
]

Ratio = [
    'HomeOwnerFlag',         
    'NumberCarsOwned',
    'NumberChildrenAtHome',
    'YearlyIncome',
    'AvgMonthSpend'
]

Interval = [
]

print("Nominal: ", Nominal)
print("Ordinal: ", Ordinal)
print("Ratio: ", Ratio)
print("Interval: ", Interval)


Nominal:  ['CountryRegionName', 'Occupation', 'Gender', 'MaritalStatus']
Ordinal:  ['Education']
Ratio:  ['HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome', 'YearlyIncome', 'AvgMonthSpend']
Interval:  []


Part II: Data Preprocessing and Transformation  
  
Depending on the data type of each attribute, transform each object from your preprocessed data.  
Use all the data rows (~= 18000 rows) with the selected features as input to apply all the tasks below, do not perform each task on the smaller data set that you got from your random sampling result. 


(a) Handling Null values 


In [37]:
import pandas as pd
from sklearn.impute import SimpleImputer

# Numeric & Categorical Columns
numeric_cols = ["Age", "YearlyIncome", "NumberCarsOwned", "NumberChildrenAtHome", "TotalChildren", "AvgMonthSpend"]
categorical_cols = ["Gender", "MaritalStatus", "Education", "Occupation", "HomeOwnerFlag"]

# Missing numeric values -> Using Median
num_imputer = SimpleImputer(strategy="median")
df[numeric_cols] = num_imputer.fit_transform(df[numeric_cols])

# Missing categorical values -> Using Mode
cat_imputer = SimpleImputer(strategy="most_frequent")
df[categorical_cols] = cat_imputer.fit_transform(df[categorical_cols])

# Drop row if BikeBuyer is missing
df = df.dropna(subset=["BikeBuyer"])

print("✅ Missing values handled!")
print(df.isnull().sum())

✅ Missing values handled!
CustomerID                  0
Title                   18260
FirstName                   0
MiddleName               7789
LastName                    0
Suffix                  18358
AddressLine1                0
AddressLine2            18050
City                        0
StateProvinceName           0
CountryRegionName           0
PostalCode                  0
PhoneNumber                 0
BirthDate                   0
Education                   0
Occupation                  0
Gender                      0
MaritalStatus               0
HomeOwnerFlag               0
NumberCarsOwned             0
NumberChildrenAtHome        0
TotalChildren               0
YearlyIncome                0
LastUpdated                 0
BikeBuyer                   0
AvgMonthSpend               0
Age                         0
dtype: int64


(b) Normalization 


In [38]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[["Age", "YearlyIncome", "AvgMonthSpend"]] = scaler.fit_transform(
    df[["Age", "YearlyIncome", "AvgMonthSpend"]]
)

print(df[["Age", "YearlyIncome", "AvgMonthSpend"]].head())


        Age  YearlyIncome  AvgMonthSpend
0  0.183099      0.496842       0.324210
1  0.408451      0.489453       0.425201
2  0.211268      0.536172       0.470977
3  0.323944      0.317083       0.605474
4  0.366197      0.231958       0.533742


(c) Discretization (Binning) on Continuous attributes or Categorical Attributes with too many different
values 


In [14]:
# Age 
df["AgeGroup"] = pd.cut(df["Age"],
                        bins=[0, 0.25, 0.5, 0.75, 1],  
                        labels=["Young", "Middle", "Mature", "Senior"])

# Income 
df["IncomeGroup"] = pd.qcut(df["YearlyIncome"], q=3, labels=["Low", "Medium", "High"])

print(df[["Age", "AgeGroup", "YearlyIncome", "IncomeGroup"]].head())


        Age AgeGroup  YearlyIncome IncomeGroup
0  0.183099    Young      0.496842      Medium
1  0.408451   Middle      0.489453      Medium
2  0.211268    Young      0.536172        High
3  0.323944   Middle      0.317083      Medium
4  0.366197   Middle      0.231958         Low


(d) Standardization/Normalization

In [15]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
df[["Age", "YearlyIncome"]] = std_scaler.fit_transform(
    df[["Age", "YearlyIncome"]]
)

print(df[["Age", "YearlyIncome"]].head())


        Age  YearlyIncome
0 -0.542849      0.298555
1  0.876937      0.271180
2 -0.365376      0.444261
3  0.344517     -0.367401
4  0.610727     -0.682765


(e) Binarization (One Hot Encoding) 

In [16]:
df_encoded = pd.get_dummies(
    df,
    columns=["Gender", "MaritalStatus", "Education", "Occupation", "AgeGroup", "IncomeGroup"],
    drop_first=True  
)

print(df_encoded.head())


   CustomerID Title FirstName MiddleName  LastName Suffix  \
0       21173   NaN      Chad          C      Yuan    NaN   
1       13249   NaN      Ryan        NaN     Perry    NaN   
2       29350   NaN     Julia        NaN  Thompson    NaN   
3       13503   NaN  Theodore        NaN     Gomez    NaN   
4       22803   NaN  Marshall          J      Shan    NaN   

             AddressLine1 AddressLine2         City    StateProvinceName  ...  \
0      7090 C. Mount Hood          NaN   Wollongong      New South Wales  ...   
1     3651 Willow Lake Rd          NaN      Shawnee     British Columbia  ...   
2  1774 Tice Valley Blvd.          NaN  West Covina           California  ...   
3         2103 Baldwin Dr          NaN    Liverpool              England  ...   
4         Am Gallberg 234          NaN        Werne  Nordrhein-Westfalen  ...   

  Education_Partial High School Occupation_Management Occupation_Manual  \
0                         False                 False             False

Part III: Calculating Proximity /Correlation Analysis of two features  
  
Make sure each attribute is transformed in a same scale for numeric attributes and Binarization for each
nominal attribute, and each discretized numeric attribute to standardization. Make sure to apply a correct
similarity measure for nominal (one hot encoding)/binary attributes and numeric attributes respectively. 

(a) Calculate Similarity in Simple Matching, Jaccard Similarity, and Cosine Similarity between two
following objects of your transformed input data. 

In [34]:
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

obj1 = df.iloc[0]
obj2 = df.iloc[1]

numeric_cols = ["Age", "YearlyIncome", "NumberCarsOwned", "NumberChildrenAtHome", "TotalChildren"]
categorical_cols = ["Gender", "MaritalStatus", "Education", "Occupation", "HomeOwnerFlag"]


df_cat = pd.get_dummies(df[categorical_cols])

obj1_cat = df_cat.iloc[0].values
obj2_cat = df_cat.iloc[1].values


obj1_num = df[numeric_cols].iloc[0].values.reshape(1, -1)
obj2_num = df[numeric_cols].iloc[1].values.reshape(1, -1)



#  Simple Matching 
smc = (obj1_cat == obj2_cat).sum() / len(obj1_cat)

#  Jaccard Similarity
jaccard = jaccard_score(obj1_cat, obj2_cat, average="binary")

#  Cosine Similarity
cosine = cosine_similarity(obj1_num, obj2_num)[0][0]

print("Simple Matching Coefficient (SMC):", smc)
print("Jaccard Similarity:", jaccard)
print("Cosine Similarity:", cosine)


Simple Matching Coefficient (SMC): 0.875
Jaccard Similarity: 0.6666666666666666
Cosine Similarity: 0.752249563659676


(b) Calculate Correlation between two features Commute Distance and Yearly Income

In [40]:
# As Commute Distance is not present in the dataset
# Calculating Correlation between Yearly Income & Bike Buyer
corr = df["YearlyIncome"].corr(df["BikeBuyer"])

print("Correlation between Yearly Income and BikeBuyer:", corr)

Correlation between Yearly Income and BikeBuyer: 0.24945696278096902
